# TopGPT Training Notebook

In [ ]:
import torch
import os
from tqdm import tqdm
from src.config import GPTConfig
from src.gpt import GPTLanguageModel

torch.manual_seed(1337)

In [ ]:
def load_data(config):
    with open(config.data_path, "r", encoding="utf-8") as f:
        text = f.read()
    
    # Character-level tokenization
    chars = sorted(list(set(text)))
    config.vocab_size = len(chars)
    
    # Create mappings
    stoi = {ch: i for i, ch in enumerate(chars)}
    itos = {i: ch for i, ch in enumerate(chars)}
    
    # Define encoding/decoding functions
    encode = lambda s: [stoi[c] for c in s]
    decode = lambda l: ''.join([itos[i] for i in l])
    
    # Train and validation split
    data = torch.tensor(encode(text), dtype=torch.long)
    n = int(0.9 * len(data))
    train_data = data[:n]
    val_data = data[n:]
    
    return train_data, val_data, encode, decode

def get_batch(config, data_split, train_data, val_data):
    data = train_data if data_split == 'train' else val_data
    ix = torch.randint(len(data) - config.block_size, (config.batch_size,))
    x = torch.stack([data[i:i+config.block_size] for i in ix])
    y = torch.stack([data[i+1:i+config.block_size+1] for i in ix])
    x, y = x.to(config.device), y.to(config.device)
    return x, y

@torch.no_grad()
def estimate_loss(config, model, train_data, val_data):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(config.eval_iters)
        for k in range(config.eval_iters):
            X, Y = get_batch(config, split, train_data, val_data)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
config = GPTConfig()
print(f"Using device: {config.device}")

In [ ]:
# Load data
train_data, val_data, encode, decode = load_data(config)
print(f"Vocabulary size: {config.vocab_size}")

In [ ]:
os.makedirs(os.path.dirname(config.model_save_path), exist_ok=True)

In [ ]:
# Initialize model
model = GPTLanguageModel(config)
model.to(config.device)
print(f"Number of parameters: {sum(p.numel() for p in model.parameters())/1e6:.2f}M")

optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate)

In [ ]:
# Training loop
for iter in tqdm(range(config.max_iters), desc="Training"):
    if iter % config.eval_interval == 0 or iter == config.max_iters - 1:
        losses = estimate_loss(config, model, train_data, val_data)
        print(f"Step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    
    xb, yb = get_batch(config, 'train', train_data, val_data)
    logits, loss = model(xb, yb)
    
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

model.save(config.model_save_path)
print(f"Model saved to {config.model_save_path}")

In [ ]:
context = torch.zeros((1, 1), dtype=torch.long, device=config.device)
generated_text = decode(model.generate(context, max_new_tokens=500)[0].tolist())
print(generated_text)

# Save longer generated text to file
with open(config.result_file, 'w') as f:
    longer_text = decode(model.generate(context, max_new_tokens=10000)[0].tolist())
    f.write(longer_text)

print(f"Longer generated text saved to {config.result_file}")

# Done